In [1]:
import numpy as np
from astropy.io import fits

## Read FITS files

Print the metadata information of the image

In [2]:
hdulist = fits.open('data/images/image0.fits')

In [3]:
hdulist.info()

Filename: data/images/image0.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       7   (200, 200)   float64   
